# Load and Cleansing of Training Data

In [608]:
import pandas as pd
import fnmatch
import os
import csv
import codecs
import boto3
import io
from io import StringIO

csv_buffer = StringIO()

## Load Data from aws S3

In [609]:
!ln -s /Users/matthiashugli/Virtualenvs/youth-base/youth-base/config.py config.py
from config import s3


ln: config.py: File exists


In [610]:
### Monthly data is stored to subfolder in S3 Bucket, read and store filenames
bucket_name = s3.Bucket('training-minutes')

bucket_list = []
for file in bucket_name.objects.filter(Prefix = 'rawdata/'):
    file_key = file.key
    if file_key.find('.csv') != -1:
        bucket_list.append(file.key)
print(len(bucket_list))

2


In [611]:
### Read all files in Bucket and return as a DataFrame
df = pd.DataFrame()
for file_name in bucket_list:
    obj = s3.Object(bucket_name.name, file_name)
    data = obj.get()['Body'].read()
    file = pd.read_csv(io.BytesIO(data), header=1, delimiter=',', low_memory=False)
    file.insert(0, 'filename', file_name)

    df = df.append(file)

### Save DataFrame to S3 as CSV
df.to_csv(csv_buffer)
s3.Object(bucket_name.name, 'staging_trainings.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'G1WYJ807DEJPDHC6',
  'HostId': 'narIgn7A7aqOg51nDGYHA6wsrfnPN/EuapsQ8sfr0c1V0Cxqy9ZrY8jhO/d3K1aBMdp0L3kc5FA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'narIgn7A7aqOg51nDGYHA6wsrfnPN/EuapsQ8sfr0c1V0Cxqy9ZrY8jhO/d3K1aBMdp0L3kc5FA=',
   'x-amz-request-id': 'G1WYJ807DEJPDHC6',
   'date': 'Mon, 29 Mar 2021 04:31:42 GMT',
   'etag': '"30c0d8b606af83d2d6957bdb7b8c76e1"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"30c0d8b606af83d2d6957bdb7b8c76e1"'}

## Cleanse Training Data

In [612]:
### Properly name columns and broad data table
df = df.drop(df.columns[[3]], axis=1)
df = df.rename(columns={'Unnamed: 0': 'team', 'Unnamed: 1': 'keeper'})
unpivot_df = pd.melt(df, id_vars=['filename', 'team', 'keeper'], var_name='topic', value_name='value')

In [613]:
### Set numeric values and rename value columns for pivot
trainings = unpivot_df

trainings['value'] = pd.to_numeric(trainings.value, errors='coerce')

trainings['topic'] = trainings.topic.apply(lambda x: x.replace('.1', '.trainings') if fnmatch.fnmatch(x, '*.1') else x + ".duration")

### Pivot DataFrame for all categories and create two columns for duration in minutes and number of trainings
trainings[['topic', 'entity']] = trainings.topic.apply(lambda x: pd.Series(str(x).split('.')))
trainings = trainings.pivot(index=['filename', 'team', 'keeper', 'topic'], columns='entity', values='value').reset_index()
trainings = trainings.dropna(subset=['duration', 'trainings'])

In [614]:
### Remove and rename club and skill details
trainings[['team', 'club']] = trainings.team.str.split(' BSC', 1, expand=True)
trainings[['team', 'club']] = trainings.team.str.split(' YB', 1, expand=True)
trainings[['category', 'skills']] = trainings.topic.apply(lambda x: pd.Series(str(x).split(':')) if fnmatch.fnmatch(x, ':') else pd.Series(str(x).split(' ', 1)))
trainings['category'] = trainings.category.str.replace(':', '')

In [615]:
# Date cleansing
trainings[['file', 'type']] = trainings.filename.apply(lambda x: pd.Series(str(x).split('.')))
trainings['yearmonth'] = trainings.file.str[-6:]
trainings['date'] = pd.to_datetime(trainings.yearmonth, format='%Y%m', errors='coerce').dropna()
trainings['year'] = pd.DatetimeIndex(trainings['date']).year
trainings['month'] = trainings.date.dt.month_name()

In [616]:
### Drop columns and reorder DataFrame
trainings['team'] = trainings.team.apply(lambda x: 'U21' if fnmatch.fnmatch(x, '1. Mannschaft') else x)
trainings = trainings.drop(columns=['filename', 'topic', 'club', 'file', 'type']).reset_index()
trainings = trainings.reindex(columns=['team', 'keeper', 'date', 'year', 'yearmonth', 'month', 'category', 'skills', 'trainings', 'duration'])

In [619]:
### Modify training unites as int
trainings['trainings'] = trainings.trainings.astype(int)

### Divide duration in mintues by 1440 to display hh:mm in report
trainings['duration'] = trainings.duration / 1440

## Export Cleansed DataFrame to aws S3

In [620]:
trainings_export = trainings
trainings_export.to_csv('cleansing_trainings.csv', index=False)
trainings.to_csv(csv_buffer)
s3.Object(bucket_name.name, 'cleansing_trainings.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'VNKW0RHVS6AH5RV6',
  'HostId': '7lEqzT9qdS8d0QLXojiPWUpfaciLrPBOt3MoqzwrENVMhAsw6qTpbs95ytG5sGDTFCDaETKzfWA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '7lEqzT9qdS8d0QLXojiPWUpfaciLrPBOt3MoqzwrENVMhAsw6qTpbs95ytG5sGDTFCDaETKzfWA=',
   'x-amz-request-id': 'VNKW0RHVS6AH5RV6',
   'date': 'Mon, 29 Mar 2021 04:42:20 GMT',
   'etag': '"92e2ecb012f67066c0b8b508509ab61d"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"92e2ecb012f67066c0b8b508509ab61d"'}

In [603]:
# Outdated!!! Read from local files, however files are stored on S3 from March 2021

path = '/Users/matthiashugli/Virtualenvs/youth-base/youth-base/training-analysis/data'

data = [os.path.join(dirpath, f) for dirpath, dirnames, files in os.walk(path) for f in fnmatch.filter(files, '*.xlsx')]

df = pd.DataFrame()
for filename in data:
    file = pd.read_excel(filename, sheet_name='s1_core_trm_player_unit_skills', index_col=None, header=1)
